In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler

data = pd.read_csv("하이닉스 scaledata1.csv")


In [4]:
pd.set_option('display.max_columns', None) 
data

,Unnamed: 0,날짜,시가,고가,저가,종가,거래량,등락률,EMA_short,EMA_long,MACD,MACD_Signal,RSI,RSI_Signal,SMA_5,SMA_20,SMA_60,SMA_120,SMA_240,EMA_5,EMA_20,EMA_60,EMA_120,EMA_240,SMA_5_minus_SMA_20,SMA_5_minus_SMA_60,SMA_5_minus_SMA_120,SMA_5_minus_SMA_240,SMA_20_minus_SMA_60,SMA_20_minus_SMA_120,SMA_20_minus_SMA_240,SMA_60_minus_SMA_120,SMA_60_minus_SMA_240,SMA_120_minus_SMA_240,EMA_5_minus_EMA_20,EMA_5_minus_EMA_60,EMA_5_minus_EMA_120,EMA_5_minus_EMA_240,EMA_20_minus_EMA_60,EMA_20_minus_EMA_120,EMA_20_minus_EMA_240,EMA_60_minus_EMA_120,EMA_60_minus_EMA_240,EMA_120_minus_EMA_240,[일]사모펀드_수량,[일]기타법인_수량,[일]국가지자체_수량,[일]연기금 등_수량,[일]기타금융_수량,[일]은행_수량,[일]투신_수량,[일]보험_수량,[일]금융투자_수량,[일]기관_수량,[일]외국인_수량,[일]개인_수량,[일]프로그램순매수(종목)
0,0,2014-03-21,-0.851149,-0.834894,-0.835520,-0.821074,15.208712,2.075296,-1.267659,-1.263866,-0.300861,-0.253689,-0.227448,-0.991544,-1.282880,-1.252255,-1.267544,-1.336290,-1.426000,-1.269214,-1.263787,-1.288239,-1.346471,-1.415729,-0.447556,-0.281577,-0.096828,0.000013,-0.095401,0.052103,0.111539,0.175187,0.179563,0.139827,-0.274761,-0.179159,-0.060667,0.002842,-0.089746,0.025411,0.070493,0.142700,0.140750,0.128242,0.480683,-1.268639,19570.0,-0.118498,0.307563,4.527523,3.981942,2.029861,0.183384,1.331608,1.374596,-1.928576,1.100531
1,1,2014-03-24,-0.824176,-0.833908,-0.823411,-0.832008,14.884212,-0.671306,-1.265633,-1.263455,-0.274031,-0.260423,-0.436816,-0.898675,-1.273775,-1.253367,-1.266920,-1.334111,-1.424583,-1.263467,-1.263127,-1.287436,-1.345115,-1.414262,-0.348618,-0.242220,-0.075148,0.017588,-0.106455,0.040905,0.105802,0.166579,0.177574,0.143218,-0.211138,-0.151854,-0.044095,0.014948,-0.090754,0.022633,0.068459,0.138380,0.138471,0.127844,0.095632,-2.803278,-2030.0,0.238107,0.000000,-1.894648,-0.586656,-0.748527,0.346853,0.043044,-0.079842,0.107574,-1.931517
2,2,2014-03-25,-0.834166,-0.843765,-0.833502,-0.842942,14.725485,-0.680446,-1.266594,-1.264378,-0.274841,-0.265984,-0.845870,-0.923591,-1.265926,-1.255036,-1.266459,-1.332085,-1.423317,-1.265396,-1.264198,-1.287250,-1.344088,-1.412968,-0.256530,-0.207949,-0.056880,0.032652,-0.120061,0.028260,0.099069,0.157892,0.175528,0.146580,-0.222019,-0.163414,-0.054749,0.006506,-0.101169,0.013485,0.061674,0.131553,0.134365,0.126329,0.072136,-0.047484,-1340.0,-0.280247,0.682231,-6.599847,1.069135,-0.588008,0.388900,0.176883,-0.702910,0.484881,0.324422
3,3,2014-03-26,-0.837163,-0.841794,-0.830474,-0.827038,14.739945,0.911733,-1.263516,-1.263337,-0.240902,-0.263194,-0.381208,-0.830827,-1.257135,-1.256387,-1.265156,-1.329808,-1.421980,-1.258303,-1.262742,-1.286211,-1.342632,-1.411451,-0.158353,-0.173072,-0.036445,0.049683,-0.136897,0.015895,0.092938,0.152649,0.175683,0.150695,-0.151424,-0.129970,-0.033551,0.022166,-0.097453,0.013860,0.061910,0.128405,0.132845,0.126287,-0.081043,-0.083852,-910.0,-0.098220,0.000000,-0.015291,0.290529,1.743246,0.036680,0.237142,0.252018,-0.352650,1.313001
4,4,2014-03-27,-0.838162,-0.841794,-0.832492,-0.840954,14.747617,-0.850505,-1.264316,-1.264032,-0.242786,-0.261363,-0.431351,-0.768776,-1.252426,-1.259248,-1.264586,-1.327643,-1.420649,-1.260906,-1.263546,-1.285958,-1.341590,-1.410151,-0.085291,-0.153818,-0.028214,0.057567,-0.158835,-0.001416,0.083218,0.143806,0.173772,0.154376,-0.174003,-0.145582,-0.046747,0.011868,-0.106195,0.005845,0.055913,0.122023,0.128986,0.124835,0.127193,0.994903,-31340.0,0.373194,-0.528450,3.006881,-0.458782,-0.578367,0.407798,0.176055,-1.613751,1.214745,-2.218120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,2457,2024-03-19,1.632368,1.590931,1.590313,1.594433,15.240856,-1.150497,2.668259,2.563740,2.213899,2.958017,0.309211,0.538093,2.644715,2.653537,2.265627,2.039794,1.693281,2.652132,2.613150,2.354529,2.136179,1.863659,0.233594,2.131085,2.254595,2.517169,2.836168,2.534317,2.651253,1.640320,1.950770,1.713819,0.914702,2.151111,2.455469,2.677485

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier


# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
data['next_day_return'] = data['등락률'].shift(-1)
data['target'] = data['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
data.dropna(inplace=True)  # 마지막 행 삭제

In [8]:
!pip install tqdm

In [11]:

X = data.drop(['날짜', 'next_day_return', 'target'], axis=1)
y = data['target']


In [12]:
import logging
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

# 데이터 로드 및 준비 (가상의 코드로, 실제 데이터셋에 맞게 조정해야 함)
# file_path = 'path_to_your_data.csv'
# data = pd.read_csv(file_path)
# X = data.drop(['날짜', '등락률', 'next_day_return', 'target'], axis=1)
# y = data['target']

# SequentialFeatureSelector 실행
sfs = SequentialFeatureSelector(RandomForestClassifier(), 
                                n_features_to_select=15,
                                direction='forward').fit(X, y)

# 선택된 피처 출력
selected_features_fwd = X.columns[sfs.get_support()]
print("Selected features by forward selection:", selected_features_fwd)

Selected features by forward selection: Index(['Unnamed: 0', '등락률', 'RSI', 'RSI_Signal', 'SMA_5_minus_SMA_240',
       'SMA_20_minus_SMA_120', 'EMA_5_minus_EMA_20', 'EMA_5_minus_EMA_120',
       '[일]국가지자체_수량', '[일]은행_수량', '[일]투신_수량', '[일]보험_수량', '[일]외국인_수량',
       '[일]개인_수량', '[일]프로그램순매수(종목)'],
      dtype='object')
